
# <span style="color:gray">ipyrad-analysis toolkit:</span> bpp (alg 00)

The ipyrad-analysis `bpp` tool w/ algorithm 00 can be used infer parameters on a fixed species tree. Please see the [full BPP documentation](https://github.com/bpp/bpp/blob/master/bppDOC.pdf) for details about BPP analyses. The ipyrad-analysis wrapper is intended to make it easy to auomate many BPP analyses on large RAD-seq datasets. 


### Load packages

In [1]:
# conda install ipyrad toytree ipcoal -c conda-forge -c bioconda
# conda install bpp -c eaton-lab

In [2]:
import ipyrad.analysis as ipa
import pandas as pd
import numpy as np
import toytree
import toyplot
import ipcoal

### Simulate data under a specified model
We are interested in estimating the Ne and divergence time values on this tree.

In [3]:
# generate a tree with divergence times in generations
TREE = toytree.rtree.unittree(ntips=5, treeheight=3e6, seed=123)

# set larger Ne on one lineage than the other
TREE = TREE.set_node_values("Ne", {i: 5e5 for i in (0, 1, 2, 5, 6)}, default=2e5)

# draw in 'p' style to show Ne
TREE.draw(ts='p', node_hover=True);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="ta1ff80e6acec48709d2b79b121dd3a40" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
dist: 1000000.0000
support: 100.0000
height: 0.0000
name: r0
Ne: 500000.0000 0 idx: 1
dist: 1000000.0000
support: 100.0000
height: 0.0000
name: r1
Ne: 500000.0000 1 idx: 2
dist: 2000000.0000
support: 100.0000
height: 0.0000
name: r2
Ne: 500000.0000 2 idx: 3
dist: 2000000.0000
support: 100.0000
height: 0.0000
name: r3
Ne: 200000.0000 3 idx: 4
dist: 2000000.0000
support: 100.0000
height: 0.0000
name: r4
Ne: 200000.0000 4 idx: 5
dist: 1000000.0000
support: 100.0000
height: 1000000.0000
name: 5
Ne: 500000.0000 5 idx: 6
dist: 1000000.0000
support: 100.0000
height: 2000000.0000
name: 6
Ne: 500000.0000 6 idx: 7
dist: 1000000.0000
support: 100.0000
height: 2000000.0000
name: 7
Ne: 200000.0000 7 idx: 8
dist: 1000000.0000
support: 100.0000
height: 3000000.0000
name: 8
Ne: 200000.0000 8 r0 r1 r2 r3 r4 0 1500000 3000000

In [4]:
# create a simulation model for this tree/network with 3 diploids per spp.
model = ipcoal.Model(tree=TREE, nsamples=6, mut=1e-8)

In [5]:
# simulate N loci
model.sim_loci(nloci=1000, nsites=150)

# write result to a database file
model.write_loci_to_hdf5(name="test-bpp", outdir="/tmp", diploid=True)

wrote 1000 loci to /tmp/test-bpp.seqs.hdf5


### *True* measurements that will inform our priors
In BPP we can set a prior on the percent divergence $\tau$ (pronounced Tau) between the root and tips of the tree. When combined with an estimate of the per-site-per-generation mutation rate ($\mu$) and generation time (g) this can be converted into units of years. In our `ipcoal` simulation we actually have access to the root ancestral sequence as well as the tip sequences, so we can measure the true percent sequence divergence.

In [6]:
# concatenate seqs into single alignment
concat = np.concatenate(model.seqs, axis=1)

# get ancestral root node sequence
rootseq = np.concatenate(model.ancestral_seq)

# what is sequence divergence relative to root?
np.sum(rootseq != concat[0]) / rootseq.size

0.03360666666666667

In addition to setting a prior on Tau we must also set a prior on $\theta$ (pronounced theta), the population mutation rate ($\theta$=4Ne$\mu$). Here we know the Ne and $\mu$ used in the simulation scenario, so again we can calculate it exactly. For real data we will not know these prior values precisely, and so we will want to put a wide prior to incorporate our uncertainty. 

In [13]:
# theta in simulation
4 * TREE.treenode.Ne * 1e-8

0.008

### BPP analysis setup

In [8]:
SEQS = "/tmp/test-bpp.seqs.hdf5"

In [9]:
IMAP = {
    "r0": ["r0-0", "r0-1", "r0-2"],
    "r1": ["r1-0", "r1-1", "r1-2"],
    "r2": ["r2-0", "r2-1", "r2-2"],
    "r3": ["r3-0", "r3-1", "r3-2"],
    "r4": ["r4-0", "r4-1", "r4-2"],
}

In [10]:
# create a bpp object to run algorithm 00
tool = ipa.bpp(
    name="test-bpp",
    workdir="/tmp",
    data=SEQS,
    guidetree=TREE,
    imap=IMAP,
    infer_sptree=0,
    infer_delimit=0,
    maxloci=1000,
    burnin=2e3,
    nsample=2e4,
    sampfreq=5,
)

### The parameters of the object

In [11]:
tool.kwargs

{'binary': '/tmp/bpp-4.1.4-linux-x86_64/bin/bpp',
 'infer_sptree': 0,
 'infer_delimit': 0,
 'infer_delimit_args': (0, 2),
 'speciesmodelprior': 1,
 'seed': 12345,
 'burnin': 2000,
 'nsample': 20000,
 'sampfreq': 5,
 'thetaprior': (3, 0.002),
 'tauprior': (3, 0.002),
 'phiprior': (1, 1),
 'usedata': 1,
 'cleandata': 0,
 'finetune': (0.01, 0.02, 0.03, 0.04, 0.05, 0.01, 0.01),
 'copied': False}

### Set priors and plot priors of transformed values

Here we will enter our expectations on the generation time and mutation rate of our simulated organisms so that we can toggle the theta and tau priors until the resulting prior distributions on Ne and T seem reasonable. 

In our simulation the root divergence time was 3M generations. Assuming a generation time of 1 and prior on root tau of ~3% (as we measured above) yields a prior on the root divergence time with 95% CI from about 0.5-6.5 Ma. So this is a pretty good but not highly informative prior. 

Similarly, our simulation used Ne=200K or 500K on different lineages. If we assume a wide prior on the per-site per-generation mutation rate ($\mu$) with highest density of 1e-8 (as used in our simulations) and a prior on $\theta$ with highest density at about 0.01 (we calculated 0.008 from the simulated data above) yields a 95% CI on Ne that is again accurate but not highly informative. 

These priors incorporate *estimated* knowledge about our organisms, by describing gentime and mutrate as distributions, and puts informative but wide priors on the parameters we hope to infer (theta and tau). 


In [14]:
# toggle these prior settings 
tool.kwargs["thetaprior"] = (2.5, 0.004)
tool.kwargs["tauprior"] = (3, 0.01)

# draw the distributions when using our assumptions for transforming
tool.draw_priors(
    gentime_min=0.99, gentime_max=1.01, 
    mutrate_min=5e-9, mutrate_max=2e-8,
);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t2b4412efcca14f2289d8399871ea33b3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 925.0 300.0" width="925.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="50.0,250.0 52.1873294844651,250.0 54.374658968930206,250.0 56.56198845339531,250.0 58.74931793786042,250.0 60.936647422325514,250.0 63.12397690679062,250.0 65.31130639125573,250.0 67.49863587572082,250.0 69.68596536018593,250.0 71.87329484465103,250.0 74.06062432911614,250.0 76.24795381358123,250.0 78.43528329804633,250.0 80.62261278251145,250.0 82.80994226697655,250.0 84.99727175144164,250.0 87.18460123590677,250.0 89.37193072037186,250.0 91.55926020483693,250.0 93.74658968930206,250.0 95.93391917376717,250.0 98.12124865823228,250.0 100.30857814269737,250.0 102.49590762716247,250.0 104.68323711162759,250.0 106.87056659609266,250.0 109.05789608055778,250.0 111.24522556502288,250.0 113.43255504948799,250.0 115.61988453395307,250.0 117.8072140184182,250.0 119.99454350288329,250.0 122.18187298734838,250.0 124.36920247181348,250.0 126.55653195627859,250.0 128.7438614407437,250.0 130.9311909252088,250.0 133.11852040967392,250.0 135.30584989413904,250.0 137.4931793786041,250.0 139.6805088630692,250.0 141.8678383475343,250.0 144.05516783199943,250.0 146.24249731646455,250.0 148.42982680092965,250.0 150.61715628539474,250.0 152.80448576985984,250.0 154.99181525432493,250.0 157.17914473879003,250.0 159.36647422325515,250.0 161.55380370772025,250.0 163.74113319218532,250.0 165.92846267665044,250.0 168.1157921611156,250.0 170.30312164558063,250.0 172.49045113004576,250.0 174.67778061451085,250.0 176.86511009897595,250.0 179.05243958344107,250.0 181.2397690679062,250.0 183.42709855237132,250.0 185.61442803683642,250.0 187.80175752130145,250.0 189.98908700576658,250.0 192.17641649023167,250.0 194.36374597469683,250.0 196.55107545916192,250.0 198.73840494362705,250.0 200.9257344280921,250.0 203.11306391255718,250.0 205.30039339702228,250.0 207.48772288148737,250.0 209.67505236595252,250.0 211.86238185041762,250.0 214.04971133488274,250.0 216.2370408193478,250.0 218.4243703038129,250.0 220.61169978827803,250.0 222.79902927274316,250.0 224.9863587572082,250.0 227.17368824167335,250.0 229.36101772613847,250.0 231.54834721060357,250.0 233.73567669506866,250.0 235.92300617953373,250.0 238.11033566399885,250.0 240.29766514846395,250.0 242.48499463292904,250.0 244.67232411739417,250.0 246.85965360185926,250.0 249.04698308632436,250.0 251.23431257078946,250.0 253.42164205525455,250.0 255.60897153971965,250.0 257.7963010241848,250.0 259.98363050864987,250.0 262.170959993115,250.0 264.3582894775801,250.0 266.5456189620452,250.0 266.5456189620452,245.82857621170837 264.3582894775801,245.42797222091957 262.170959993115,244.99222674123868 259.98363050864987,244.5186572230396 257.7963010241848,244.00442394058808 255.60897153971965,243.44652658220596 253.42164205525455,242.84180158743973 251.23431257078946,242.18692036743545 249.04698308632436,241.47838855677017 246.85965360185926,240.71254645721237 244.67232411739417,239.88557084613817 242.48499463292904,238.99347833444662 240.29766514846395,238.03213047059734 238.11033566399885,236.9972407986054 235.92300617953373,235.88438408820605 233.73567669506866,234.68900796464467 231.54834721060357,233.40644717331193 229.36101772613847,232.03194072035902 227.17368824167335,230.56065213406546 224.9863587572082,228.98769309264077 222.79902927274316,227.3081506618176 220.61169978827803,225.51711837951035 218.4243703038129,223.60973141438646 216.2370408193478,221.58120600984623 214.04971133488274,219.42688340398269 211.86238185041762,217.14227838897585 209.6750523

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t36ed86df0ce64fa7a44a415843f71ff8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 925.0 300.0" width="925.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="50.0,250.0 52.27272727272745,250.0 54.545454545454895,250.0 56.81818181818138,250.0 59.09090909090882,250.0 61.36363636363627,250.0 63.636363636363726,250.0 65.90909090909118,250.0 68.18181818181861,250.0 70.4545454545451,250.0 72.72727272727255,250.0 75.0,250.0 77.27272727272745,250.0 79.5454545454549,250.0 81.81818181818139,250.0 84.09090909090882,250.0 86.36363636363627,250.0 88.63636363636371,250.0 90.90909090909116,250.0 93.18181818181863,250.0 95.4545454545451,250.0 97.72727272727255,250.0 100.0,250.0 102.27272727272745,250.0 104.5454545454549,250.0 106.81818181818137,250.0 109.09090909090881,250.0 111.36363636363627,250.0 113.63636363636373,250.0 115.90909090909116,250.0 118.18181818181861,250.0 120.4545454545451,250.0 122.72727272727255,250.0 125.0,250.0 127.27272727272745,250.0 129.5454545454549,250.0 131.8181818181814,250.0 134.0909090909088,250.0 136.36363636363626,250.0 138.63636363636374,250.0 140.90909090909116,250.0 143.1818181818186,250.0 145.4545454545451,250.0 147.72727272727252,250.0 150.0,250.0 152.27272727272745,250.0 154.5454545454549,250.0 156.8181818181814,250.0 159.09090909090884,250.0 161.36363636363626,250.0 163.63636363636374,250.0 165.9090909090902,250.0 168.1818181818186,250.0 170.4545454545451,250.0 172.72727272727352,250.0 175.0,250.0 177.27272727272648,250.0 179.5454545454549,250.0 181.8181818181814,250.0 184.0909090909098,250.0 186.36363636363626,250.0 188.63636363636277,250.0 190.90909090909116,250.0 193.18181818181765,250.0 195.45454545454606,250.0 197.72727272727255,250.0 199.99999999999903,250.0 202.27272727272745,250.0 204.5454545454539,250.0 206.81818181818235,250.0 209.09090909090884,250.0 211.36363636363726,250.0 213.6363636363637,250.0 215.9090909090902,250.0 218.1818181818186,250.0 220.45454545454513,250.0 222.72727272727352,250.0 225.0,250.0 227.2727272727265,250.0 229.5454545454549,250.0 231.81818181818136,250.0 234.0909090909098,250.0 236.36363636363626,250.0 238.63636363636274,250.0 240.90909090909116,250.0 243.18181818181765,250.0 245.45454545454606,250.0 247.72727272727255,250.0 250.00000000000097,250.0 252.27272727272745,250.0 254.54545454545394,250.0 256.8181818181823,250.0 259.09090909090884,250.0 261.3636363636372,250.0 263.6363636363637,250.0 265.9090909090902,250.0 268.18181818181864,250.0 270.4545454545451,250.0 272.72727272727354,250.0 275.0,250.0 275.0,242.81278281447084 272.72727272727354,241.80067877208955 270.4545454545451,240.67071641213906 268.18181818181864,239.4130378685218 265.9090909090902,238.017525704654 263.6363636363637,236.4738828849191 261.3636363636372,234.77172604783328 259.09090909090884,232.90069229797447 256.8181818181823,230.85055942624584 254.54545454545394,228.6113792646309 252.27272727272745,226.17362349236396 250.00000000000097,223.52834098219137 247.72727272727255,220.66732534715393 245.45454545454606,217.5832910447992 243.18181818181765,214.27005601653804 240.90909090909116,210.7227285013 238.63636363636274,206.93789529682016 236.36363636363626,202.9138084504044 234.0909090909098,198.6505671015962 231.81818181818136,194.15029086423928 229.5454545454549,189.41728116095777 227.2727272727265,184.45816652217303 225.0,179.2820281621591 222.72727272727352,173.90050183510994 220.45454545454513,168.32785248384096 218.1818181818186,162.58101809190282 215.9090909090902,156.67961967441482 213.6363636363637,150.6459346949981 211.36363636363726,144.5048317865736 209.09090909090884,138.28366501062484 20

### Run inference
Distribute jobs in parallel and run replicate jobs that start MCMC from different random seeds. 

In [15]:
tool.run(auto=True, nreps=2, force=True)

Parallel connection | latituba: 8 cores
[locus filter] full data: 1000
[locus filter] post filter: 1000
[ipa bpp] bpp v4.1.4
[ipa.bpp] distributed 2 bpp jobs (name=test-bpp, nloci=1000)
[####################] 100% 3:36:37 | progress on all reps 
Parallel connection closed.


### Re-load results
You can load your results anytime later after your runs have finished by providing the name and workdir for the job to ipa.bpp. This allows you to mess around with plotting and comparing distributions later. You also have the option here to combine the posteriors from multiple replicate runs (that used the same data and priors), or to analyze each separately. 

In [1]:
import ipyrad.analysis as ipa

In [2]:
# reload results by creating object with a given name and workdir
tool = ipa.bpp(name="test-bpp", workdir="/tmp")

# call summarize on the tool
table, mcmc = tool.summarize_results("00", individual_results=False)

[ipa.bpp] found 2 existing result files
[ipa.bpp] summarizing algorithm '00' results
[ipa.bpp] combining mcmc files


### The "00" tables result

The main result of the "00" algorithm is a dataframe with the inferred parameters of the multispecies coalescent model. 

In [3]:
# tables is a summary of the posteriors
table

,theta_1r0,theta_2r1,theta_3r2,theta_4r3,theta_5r4,theta_6r4r3r2r1r0,theta_7r4r3,theta_8r2r1r0,theta_9r1r0,tau_6r4r3r2r1r0,tau_7r4r3,tau_8r2r1r0,tau_9r1r0,lnL
mean,0.027995,0.028212,0.025590,0.009736,0.009295,0.007839,0.005385,0.022274,0.017841,0.034456,0.024037,0.026410,0.016234,-418047.448228
median,0.027980,0.028194,0.025576,0.009733,0.009287,0.007836,0.005425,0.022171,0.017826,0.034457,0.024008,0.026409,0.016230,-418046.894000
S.D,0.000885,0.000904,0.000717,0.000304,0.000296,0.000704,0.001142,0.002391,0.001121,0.000394,0.000616,0.000444,0.000313,79.616227
min,0.024424,0.024989,0.022996,0.008656,0.008194,0.005509,0.002602,0.014451,0.013855,0.032841,0.021608,0.024757,0.015049,-418388.687000
max,0.032145,0.032099,0.028488,0.011026,0.010516,0.010827,0.009983,0.032692,0.023367,0.036106,0.026211,0.028206,0.017480,-417729.014000
2.5%,0.026300,0.026481,0.024243,0.009154,0.008728,0.006511,0.003146,0.017910,0.015693,0.033689,0.022892,0.025541,0.015626,-418202.863000
97.5%,0.029785,0.030023,0.027033,0.010342,0.009899,0.009229,0.007568,0.027284,0.020108,0.035223,0.025280,0.027283,0.016861,-417892.551000
2.5%HPD,0.026283,0.026471,0.024219,0.009152,0.008691,0.006477,0.002991,0.017773,0.015733,0.033673,0.022859,0.025528,0.015623,-418201.078000
97.5%HPD,0.029757,0.030004,0.027007,0.010338,0.009860,0.009190,0.007314,0.027099,0.020134,0.035200,0.025239,0.027268,0.016855,-417891.188000
ESS*,12262.897304,11997.844527,13891.059855,6974.445253,6525.824045,436.125051,123.289724,1897.302231,3587.614817,579.708878,147.436912,2641.537188,4378.739376,2087.556267


### Converting units from the table results
The results are often difficult to interpret without converting the units into a more easily interpretable type. For example, the theta parameters represent the population effective mutation rate (`4*Ne*u`), but we are typically more interested in just knowing `N`. Similarly, the tau parameters are in units of percent sequence divergence (u/gen) but we instead like to know the divergence time estimates in units of years, or millions of years.

To convert these units we need an estimate of the per-site per-generation mutation-rate (u) and the generation time of our organism (g). Rather than offering a point estimate it is more appropriate in the Bayesian context to provide these values a statistical distribution. This is similar to how we described our priors using a gamma or inverse-gamma distribution earlier, and how we used a distribution of u and g to check the reasonable conversion of the units.

In the example below the converted units can be compared with the true simulation scenario at the beginning of this notebook where we set the divergence times and Ne values on the tree. Among the tip-level taxa we can see that mean Ne estimates are around 2e5 or ~6e5, which is pretty accurate, and the root divergence time is at 3e6, which is correct. 

In [17]:
df = tool.transform(mcmc, 0.99, 1.01, 5e-9, 2e-8).T

In [18]:
df

,mean,median,S.D,min,max
Ne_1r0,6.16e+05,5.78e+05,2.05e+05,1.89e+05,2.55e+06
Ne_2r1,6.2e+05,5.83e+05,2.06e+05,1.9e+05,2.57e+06
Ne_3r2,5.63e+05,5.29e+05,1.87e+05,1.72e+05,2.28e+06
Ne_4r3,2.14e+05,2.01e+05,7.12e+04,6.57e+04,8.87e+05
Ne_5r4,2.04e+05,1.92e+05,6.8e+04,6.26e+04,8.47e+05
Ne_6r4r3r2r1r0,1.73e+05,1.62e+05,5.93e+04,5.47e+04,7.23e+05
Ne_7r4r3,1.2e+05,1.11e+05,4.54e+04,2.8e+04,5.7e+05
Ne_8r2r1r0,4.91e+05,4.59e+05,1.71e+05,1.45e+05,2.17e+06
Ne_9r1r0,3.92e+05,3.68e+05,1.32e+05,1.3e+05,1.71e+06
div_6r4r3r2r1r0,3.03e+06,2.85e+06,1e+06,9.51e+05,1.24e+07


### Plot posteriors
You can plot several posterior distributions on a shared axis using the `.draw_posteriors()` function. This takes a list of tuples as input, where each tuple is the (mean, variance) of a parameter estimate. You can draw the posteriors from a single analysis using this function or combine results from several different analyses if you wish. 

In [43]:
# get only the divergence time result
subd = df.loc[[i for i in df.index if "div_" in i], :]

# get results as tuple pairs with (mean, var)
tuples = [
    (i, j) for (i, j) in 
    zip(subd["mean"], subd.loc[:, "S.D"] ** 2)
]

# draw the results
c, a, m = tool.draw_posteriors(
    gamma_tuples=tuples,
    labels=subd.index,
);

# style the axes
a.y.ticks.labels.angle = -90

<svg class="toyplot-canvas-Canvas" height="350.0px" id="t45f8723ff6384bdbb6da8cc07e85a9ec" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 225.0 350.0" width="225.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="69.67116700792698,122.5806451612903 70.67221511900189,122.5806451612903 71.67326323007681,122.5806451612903 72.67431134115172,122.5806451612903 73.67535945222663,122.5806451612903 74.67640756330155,122.5806451612903 75.67745567437646,122.5806451612903 76.67850378545137,122.5806451612903 77.67955189652628,122.5806451612903 78.68060000760119,122.5806451612903 79.68164811867611,122.5806451612903 80.68269622975102,122.5806451612903 81.68374434082592,122.5806451612903 82.68479245190085,122.5806451612903 83.68584056297576,122.5806451612903 84.68688867405066,122.5806451612903 85.68793678512557,122.5806451612903 86.6889848962005,122.5806451612903 87.69003300727539,122.5806451612903 88.6910811183503,122.5806451612903 89.69212922942523,122.5806451612903 90.69317734050014,122.5806451612903 91.69422545157505,122.5806451612903 92.69527356264996,122.5806451612903 93.69632167372487,122.5806451612903 94.69736978479978,122.5806451612903 95.69841789587468,122.5806451612903 96.6994660069496,122.5806451612903 97.7005141180245,122.5806451612903 98.70156222909942,122.5806451612903 99.70261034017433,122.5806451612903 100.70365845124924,122.5806451612903 101.70470656232416,122.5806451612903 102.70575467339907,122.5806451612903 103.70680278447398,122.5806451612903 104.70785089554889,122.5806451612903 105.70889900662381,122.5806451612903 106.70994711769872,122.5806451612903 107.71099522877364,122.5806451612903 108.71204333984855,122.5806451612903 109.71309145092346,122.5806451612903 110.71413956199837,122.5806451612903 111.71518767307327,122.5806451612903 112.71623578414821,122.5806451612903 113.7172838952231,122.5806451612903 114.71833200629801,122.5806451612903 115.71938011737294,122.5806451612903 116.72042822844784,122.5806451612903 117.72147633952277,122.5806451612903 118.72252445059766,122.5806451612903 119.72357256167258,122.5806451612903 120.7246206727475,122.5806451612903 121.7256687838224,122.5806451612903 122.72671689489732,122.5806451612903 123.72776500597223,122.5806451612903 124.72881311704714,122.5806451612903 125.72986122812205,122.5806451612903 126.73090933919694,122.5806451612903 127.73195745027186,122.5806451612903 128.73300556134677,122.5806451612903 129.73405367242168,122.5806451612903 130.73510178349662,122.5806451612903 131.7361498945715,122.5806451612903 132.73719800564643,122.5806451612903 133.73824611672134,122.5806451612903 134.73929422779625,122.5806451612903 135.74034233887116,122.5806451612903 136.74139044994607,122.5806451612903 137.74243856102098,122.5806451612903 138.7434866720959,122.5806451612903 139.74453478317082,122.5806451612903 140.74558289424573,122.5806451612903 141.74663100532064,122.5806451612903 142.74767911639555,122.5806451612903 143.74872722747045,122.5806451612903 144.7497753385454,122.5806451612903 145.7508234496203,122.5806451612903 146.7518715606952,122.5806451612903 147.75291967177012,122.5806451612903 148.75396778284502,122.5806451612903 149.75501589391993,122.5806451612903 150.75606400499484,122.5806451612903 151.75711211606978,122.5806451612903 152.75816022714469,122.5806451612903 153.7592083382196,122.5806451612903 154.7602564492945,122.5806451612903 155.7613045603694,122.5806451612903 156.76235267144432,122.5806451612903 157.76340078251923,122.5806451612903 158.76444889359414,122.5806451612903 159.76549700466904,122.5806451612903 160.76654511574395,122.5806451612903 161.76759322681886,122.5806451612903 162.76864133789377,122.5806451612903 163.

### Plot results on a tree

... coming